## TODO
* Features:
    * Easy visualization of: case starts, concurrent case obligations
    * printouts/reports of parameters used to generate analyses
    * utils: determine if an event falls in an arbitrary time window; aggregate to show a weekly incidence (across days of week), vs 52 weeks/yr, 12 months/year; expected formats for activity events 
    * some clinical questions to be answered: mishaps during nights and wkends vs weekdays, per provider, or likelihood when activity (anesthesia demand) is high, blocks of time with dearth of activity for didactics
    * start and end event priorities for defining an activity instance-->needs to eventually create a timeseries (long format) and use these as start and end
    
## Next
* should there be a ConcurrencyTimeSeries object to extend Series and tack on some common functionality?
* AnesthesiaResourceAssignmentDataSet
    * Would process in long format "schedule" data 
    * Would use ProviderShift objects to interpret and translate its entries into capacity figures

In [1]:
import json
import sys
import os
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

sys.path.append("/home/tj/PycharmProjects/pytiva")
import pytiva

tests_wd = "/home/tj/PycharmProjects/pytiva/tests"
sys.path.append(tests_wd)
from tests import testconfig
data_wd = os.path.join(tests_wd, testconfig.WD)

PARAMS = {
    'DATE_START': pd.to_datetime('2021-10-01'),
    'DATE_END': pd.to_datetime('2021-12-31'),
    'FREQ': 'T'
}

def generate_testdata_study():
    testdata_csv_dict = {
        'ds_cases': { 'FILEPATH': os.path.join(data_wd, testconfig.TESTDATA['DS_CASES']) },
        'ds_case_meds': { 'FILEPATH': os.path.join(data_wd, testconfig.TESTDATA['DS_CASE_MEDS']) },
        'ds_case_events': { 'FILEPATH': os.path.join(data_wd, testconfig.TESTDATA['DS_CASE_EVENTS']) }
    }
    datasets = pytiva.anesthesia.datasets_from_csv_data(testdata_csv_dict)
    study = pytiva.anesthesia.AnesthesiaStudy(**datasets)
    
    # activities
    h = open(os.path.join(tests_wd, testconfig.WD, testconfig.TESTDATA['EVENT_BASED_ACTIVITY_DEFINITIONS']), "r")
    ea_definitions = json.load(h)
    h.close()
    
    study.ds_activity = pytiva.ActivityDataSet(
        pd.concat([
            pytiva.anesthesia.EventActivityDefinition(**d).apply_to_ds(study.ds_case_events)
            for d in ea_definitions
        ] + [study.ds_case_meds.to_activity_dataset(pd.to_timedelta(2, unit='T'), pd.to_timedelta(3, unit='T'))._df]
        ).reset_index(drop=True)
    )
    return study

study = generate_testdata_study()
study.limit_by_dates(dt_start = testconfig.TESTDATA_STUDY_DATES[0], dt_end = testconfig.TESTDATA_STUDY_DATES[1])
study #maybe make the propagation stuff only apply to members starting with "ds_case_"

### AnesthesiaStudy object ###
* Case data *
ds_cases['anesthesia_start'] range: 2021-10-02 07:02:20 to 2021-12-30 15:45:37
ds_cases (462 rows)
ds_case_events (8301 rows)
ds_case_meds (4033 rows)

* Activity data *
ds_activity: "medication" (4033 entries), "activity A" (404 entries), "activity B" (173 entries)
activity    activity_start
activity A  2021-10-31         129
            2021-11-30         147
            2021-12-31         128
activity B  2021-10-31          59
            2021-11-30          52
            2021-12-31          62
medication  2021-10-31        1352
            2021-11-30        1360
            2021-12-31        1321

In [2]:
# TODO: ponder, ye
# should there be a way to use the unduplication functionality to generate capacity from ads_resources?
# or is that a distinct mathematical operation?
# I think it's distinct, cause otherwise I'd need to apply capacity as a weight of sorts
# so still need to generate a timeseries index with the desired resolution, start, and end points
# and then for each entry in that index, calculate the capacity

In [3]:
# TODO: unittest for generating capacity without params
# TODO: unittest for generating capacity with start and end params
# TODO: unittest for generating capacity with frequency param
# TODO: unittest for generating capacity with start, end, and frequency params
ds_resources = pytiva.staffing.ResourceAssignmentDataSet(
    pd.read_csv(os.path.join(tests_wd, testconfig.WD, testconfig.TESTDATA['STAFFING_LONG']))
)
ps_dict = pytiva.staffing.utils.provider_shift_defs_to_kv_dict(testconfig.STAFF_SHIFT_DEFINITIONS)
ds_resources.limit_by_list('assignment', ps_dict.keys())
#ds_resources.limit_by_list('date', pd.date_range(start=PARAMS['DATE_START'], end=PARAMS['DATE_END'], freq='D'))
tsds_capacity = ds_resources.generate_capacity_tsds(ps_dict,
                                                    #start_dt=PARAMS['DATE_START'],
                                                    #end_dt=PARAMS['DATE_END'],
                                                    #freq=PARAMS['FREQ']
                                                   )
tsds_wo_path = os.path.join(tests_wd, testconfig.WD, 'testdata-ds_resources_to_tsds_capacity.csv')
tsds_capacity.to_csv(tsds_wo_path)#, index_label='ts_index')
tsds_capacity

,capacity
ts_index,
2021-07-09 07:00:00,1.0
2021-07-09 07:01:00,1.0
2021-07-09 07:02:00,1.0
2021-07-09 07:03:00,1.0
2021-07-09 07:04:00,1.0
...,...
2022-03-07 14:55:00,1.0
2022-03-07 14:56:00,1.0
2022-03-07 14:57:00,1.0


In [4]:
df_tsds = pd.read_csv(tsds_wo_path)
df_tsds.set_index('ts_index')

,capacity
ts_index,
2021-07-09 07:00:00,1.0
2021-07-09 07:01:00,1.0
2021-07-09 07:02:00,1.0
2021-07-09 07:03:00,1.0
2021-07-09 07:04:00,1.0
...,...
2022-03-07 14:55:00,1.0
2022-03-07 14:56:00,1.0
2022-03-07 14:57:00,1.0


In [7]:
df_tsds.columns

Index(['ts_index', 'capacity'], dtype='object')

In [ ]:
label = 'capacity'
cc_activity_wa = pytiva.activity.utils.concurrent_weekly_activity(tsds_capacity, label=label)
graph = sn.heatmap(cc_activity_wa, robust=True,
                  cbar_kws={
                      'label': label
                      #'ticks': [0.0, 0.5, 1.0, 1.5, 2.0, 2.5]
                           })
plt.ylabel('Minute of day')
sn.set(rc={'figure.figsize':(11.7,8.27)})
plt.show()